Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [120]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic   # provide distances between cities
import networkx as nx

from icecream import ic

import random
random.seed(42)
np.random.seed(42)

logging.basicConfig(level=logging.DEBUG)

In [121]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
#CITIES = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])
#CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
#CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
#CITIES = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])


In [122]:
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))  # matrix of distances
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [123]:
# the cost is the total of distances (for this rute is about 4100km)

def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]  # starting point has to be the arrival
    assert set(tsp) == set(range(len(CITIES)))  #all the cities inside the solution

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Greedy Algorithm

In [124]:
def forward_steps(next_cities, current_path, paths, k):
    if next_cities.size == 0 or len(current_path) == k:
        paths.append(current_path)
        return
    
    for c in range(len(next_cities)):
        new_next_cities = next_cities.copy()
        forward_steps(np.delete(new_next_cities, c), current_path + [next_cities[c]], paths, k), 
        

In [125]:
def k_forward_steps(city, visited, k):
    
    next_cities = np.where(visited == False)[0]
    paths = [] # if there aren't remaining cities the cost will still correct
    best_cost = float('inf')
    best = None

    # recursively we try the next city
    for c in range(len(next_cities)):
        new_next_cities = next_cities.copy()
        forward_steps(np.delete(new_next_cities, c), [next_cities[c]], paths, k)
    
    for p in range(len(paths)):
        cost = DIST_MATRIX[city,paths[p][0]] 
        
        for i in range(len(paths[p])-1):
            cost += DIST_MATRIX[paths[p][i],paths[p][i+1]]
        if cost < best_cost:
            best_cost = cost
            best = paths[p]
        
    return best[0]

In [126]:
# consider only the current best choice -> go to closest city

visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True  # we track the visited cities
tsp = list()   # tsp list of rute (TSP, Traveling Salesman Problem)
tsp.append(int(city))

k = 1
closest = 0

greedy_iterations = 0

while not np.all(visited):
    
    dist[:, city] = np.inf

    closest = k_forward_steps(city, visited, k) 

    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

## GENETIC ALGORITHM

In [127]:
from dataclasses import dataclass, field

@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = field(default=None)

    def __post_init__(self):
        if self.fitness is None:
            self.fitness = self.evaluate_fitness()

    def evaluate_fitness(self) -> float:
        return -float(tsp_cost(self.genome))
    
    
def fitness(individual):      
    return -float(tsp_cost(individual.genome))

In [128]:
# SELECTION STEPS:
np.random.seed(42)

def random_selection(population, n):

    
    selected_individuals = []
    
    while len(selected_individuals) < n:
        
        individual = random.choice(population)
        
        if not any(np.array_equal(individual, ind) for ind in selected_individuals):
            selected_individuals.append(individual)
   
    return selected_individuals



def weighted_fitness_selection(population, n):

    list_fitness = [individual.fitness for individual in population]
    
    # we define the probability to select parents using linear function 
    total_fitness = np.sum(list_fitness)
    probabilities = list_fitness / total_fitness

    # alternative: soft max function
    # e_x = np.exp(np.array(list_fitness)) 
    # probabilities = e_x / e_x.sum()

    # select n individuals
    selected_individuals = random.choices(population, weights=probabilities, k=n)
    return selected_individuals




In [129]:
# CROSSOVER
np.random.seed(42)

def inver_over_crossover(p1: Individual, p2: Individual):
    geno1 = p1.genome
    geno2 = p2.genome
    
    child_genome = [None] * len(geno1)
    child_genome[0] = geno1[0]
    child_genome[-1] = geno1[-1]

    # Random crossover points
    start = np.random.randint(1, len(geno1) - 1)
    end = np.random.randint(1, len(geno1) - 1)
    
    if start > end:
        start, end = end, start

    child_genome[start:end + 1] = geno1[start:end + 1]

    pos = 1
    for gene in geno2:
        if gene not in child_genome:
            while pos < len(child_genome) - 1 and child_genome[pos] is not None:
                pos += 1
            if pos < len(child_genome) - 1:
                child_genome[pos] = gene

    return Individual(np.array(child_genome))

In [130]:

np.random.seed(42)


def swap_mutation(parent):
    offspring = parent.copy()
    idx1, idx2 = random.sample(range(1,len(offspring)-2), 2)
    offspring[idx1], offspring[idx2] = offspring[idx2], offspring[idx1]
    
    return offspring


def k_next_mutations(parent):
    k=max(int(len(CITIES)/10),1)  
    offspring = parent.copy()
    index = random.sample(range(1,len(offspring)-2), 1)[0]
    max_k = min(k, len(offspring) - (index + 1)-1)
    offspring_list = []
    
    for i in range(1, max_k + 1):
        offspring = parent.copy()
        offspring[index], offspring[index + i] = offspring[index + i], offspring[index]
        offspring_list.append(offspring)

    offspring_fitness = [-tsp_cost(x) for x in offspring_list]

    best_newborn = offspring_fitness.index(max(offspring_fitness))
    
    return offspring_list[best_newborn]


     
def insert_mutation(parent):
    offspring = parent.copy()
    loci = random.sample(range(1,len(offspring)-2), 2)
    i, j = sorted(loci)
    element = offspring[j]
    offspring = np.delete(offspring, j)
    offspring = np.insert(offspring, i + 1, element)
    return offspring


def inversion_mutation(parent):
    
    offspring = parent.copy()
    loci = random.sample(range(1,len(parent)-2), 2)
    i, j = sorted(loci)
    offspring[i:j+1] = offspring[i:j+1][::-1]
    return offspring
    

In [131]:

np.random.seed(42)
random.seed(42)


NUM_GENERATIONS = 1000
POPULATION_SIZE = int(len(CITIES)/1.5)
OFFSPRING_SIZE = 3*POPULATION_SIZE

mutation_prob = [1/4, 1/4, 1/4, 1/4]
mutations = [swap_mutation, k_next_mutations, insert_mutation, inversion_mutation]
crossover_prob = [0.5, 0.5]
crossovers = [inver_over_crossover, inver_over_crossover]

mutation_steps = int(OFFSPRING_SIZE*19/20)
crossover_steps = int(OFFSPRING_SIZE*1/20)





population = list()

# as suggested by Vida Gallo the algorithm stars using greedy solution
# crate initial population by swappyng the elements of greedy solution
for i in range(POPULATION_SIZE-1):    
    new_solution = np.copy(tsp)

    #n_swaps  = random.randint(4, int(len(tsp)/5)) #vanatu: random.randint(1, 2)) 
    n_swaps  = random.randint(1,2)
    for n in range(n_swaps):
        a = random.randint(1, len(tsp)-2)
        b = random.randint(1, len(tsp)-2)
        new_solution[a], new_solution[b] = new_solution[b], new_solution[a]

    population.append(new_solution) 

population.append(tsp)

# to run evolutionary algorithm it is better to use the Individual class
population = [Individual(genome) for genome in population]




champion_solution = max(population, key=lambda individual: individual.fitness)  
best_fitness = champion_solution.fitness


In [132]:
random.seed(42)
np.random.seed(42)
ea_iterations = 0
no_improvement_it = 0 # no-improvement counter


population.sort(key=lambda i: i.fitness, reverse=True) 


for g in range(NUM_GENERATIONS):
    ea_iterations = ea_iterations + 1
    
    if no_improvement_it > 1000:
        break


    selected_parents = []
    offspring = []

    for _ in range(int(POPULATION_SIZE/3)):
        if random.random() < 0.5:
            selected_parents.append(random_selection(population, 1)[0])
        else:
            selected_parents.append(weighted_fitness_selection(population, 1)[0])
    
    for _ in range(mutation_steps):
        selected_parent = random.choice(selected_parents)
        selected_mutation = np.random.choice(mutations, p=mutation_prob)
        newborn = Individual(selected_mutation(selected_parent.genome))
        offspring.append(newborn)

    for _ in range(crossover_steps):
        selected_parent1 = random.choice(selected_parents)
        selected_parent2 = random.choice(selected_parents)
        selected_crossover = np.random.choice(crossovers, p=crossover_prob)
        newborn = selected_crossover(selected_parent1, selected_parent2)
        
        offspring.append(newborn)

    population.extend(offspring)

    elitist_population = sorted(population, key=lambda individual: individual.fitness, reverse=True)[:len(population) // 2]
    population[:len(population) // 2] = elitist_population
    popultaion = population[:POPULATION_SIZE] 

    current_best = max(population, key=lambda individual: individual.fitness)  
    if current_best.fitness > best_fitness:
        best_fitness = current_best.fitness
        no_improvement_it = 0  
    else:
        no_improvement_it += 1
    champion_solution = current_best


print('Solution cost: ', tsp_cost(champion_solution.genome))
print('Total it. (number of generations):', ea_iterations)
  
    

Solution cost:  4258.069651791474
Total it. (number of generations): 1000
